In [1]:
import docker
import time

In [2]:
client = docker.from_env()

In [3]:
with open("templates/tsp_schema/main.tsp", "r") as f:
    tsp_schema = f.read()

In [4]:
def make_command(typespec_definitions: str):
    return [
        "sh",
        "-c",
        f"echo '{typespec_definitions}' > schema.tsp && tsp compile schema.tsp --no-emit"
    ]

In [5]:
container = client.containers.run(
    "botbuild/tsp_compiler",
    #["tsp", "compile", ".", "--no-emit"],
    #make_command("type A = Int"),
    make_command(tsp_schema),
    stderr=True,
    detach=True,
    remove=True,
)
output = container.attach(stdout=True, stderr=True, stream=True, logs=True)
stdout = [line.decode("utf-8") for line in output]
for line in output:
    print(line) 
result = container.wait()

In [6]:
stdout, result

(['TypeSpec compiler v0.64.0\n\n', 'Compilation completed successfully.\n\n'],
 {'StatusCode': 0})